# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import os
import pickle
import pandas as pd
import numpy as np
import nltk
import re
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sqlalchemy import create_engine
from nltk.corpus import stopwords
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
engine = create_engine('sqlite:///database.db')
df = pd.read_sql_table('messages_classification', engine)
#df.to_csv('messages_classification.csv', index=False)
df.drop('original', axis=1, inplace=True)
df.dropna(inplace=True

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    clean_tokens =  text.apply(lambda x: x.lower()).apply(lambda x: re.sub(r'[^\w\s]', '', x))
    
    return clean_tokens
X = tokenize(df['message']).values
Y = df.iloc[:, 5:40]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('classifier', MultiOutputClassifier(RandomForestClassifier()))
])

### 5. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [8]:
# Step 5: Train the model
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
pipeline.fit(X_train, y_train)

# Predict on the testing data
y_pred = pipeline.predict(X_test)

# Convert y_test and y_pred to NumPy arrays
y_test_array = y_test.values
y_pred_array = np.array(y_pred)

# Iterate through each column and print classification report
for i, column in enumerate(Y.columns):
    report = classification_report(y_test_array[:, i], y_pred_array[:, i])
    print(f"Report for {column}:")
    print(report)
    print('-' * 50)

Report for offer:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244

--------------------------------------------------
Report for aid_related:
             precision    recall  f1-score   support

          0       0.76      0.84      0.80      3113
          1       0.73      0.61      0.66      2131

avg / total       0.75      0.75      0.74      5244

--------------------------------------------------
Report for medical_help:
             precision    recall  f1-score   support

          0       0.93      0.99      0.96      4822
          1       0.55      0.11      0.19       422

avg / total       0.90      0.92      0.90      5244

--------------------------------------------------
Report for medical_products:
             precision    recall  f1-score   support

          0       0.95      1.00      0.98      4974

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
parameters = {
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': [True],
    'classifier__estimator__max_depth': [5, 7]
}

# Perform grid search
model = GridSearchCV(pipeline, parameters, cv=3, n_jobs=-1, verbose=2)

In [11]:
model.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] classifier__estimator__max_depth=5, tfidf__ngram_range=(1, 1), tfidf__use_idf=True 
[CV] classifier__estimator__max_depth=5, tfidf__ngram_range=(1, 1), tfidf__use_idf=True 
[CV] classifier__estimator__max_depth=5, tfidf__ngram_range=(1, 1), tfidf__use_idf=True 
[CV] classifier__estimator__max_depth=5, tfidf__ngram_range=(1, 2), tfidf__use_idf=True 
[CV] classifier__estimator__max_depth=5, tfidf__ngram_range=(1, 2), tfidf__use_idf=True 
[CV] classifier__estimator__max_depth=5, tfidf__ngram_range=(1, 2), tfidf__use_idf=True 
[CV] classifier__estimator__max_depth=7, tfidf__ngram_range=(1, 1), tfidf__use_idf=True 
[CV] classifier__estimator__max_depth=7, tfidf__ngram_range=(1, 1), tfidf__use_idf=True 
[CV] classifier__estimator__max_depth=7, tfidf__ngram_range=(1, 1), tfidf__use_idf=True 
[CV] classifier__estimator__max_depth=7, tfidf__ngram_range=(1, 2), tfidf__use_idf=True 
[CV] classifier__estimator__max_depth=7, tfidf__ng

[Parallel(n_jobs=-1)]: Done   2 out of  12 | elapsed:   43.0s remaining:  3.6min


[CV]  classifier__estimator__max_depth=5, tfidf__ngram_range=(1, 1), tfidf__use_idf=True, total=  30.7s
[CV]  classifier__estimator__max_depth=7, tfidf__ngram_range=(1, 1), tfidf__use_idf=True, total=  38.4s
[CV]  classifier__estimator__max_depth=7, tfidf__ngram_range=(1, 1), tfidf__use_idf=True, total=  40.6s
[CV]  classifier__estimator__max_depth=7, tfidf__ngram_range=(1, 1), tfidf__use_idf=True, total=  39.5s
[CV]  classifier__estimator__max_depth=5, tfidf__ngram_range=(1, 2), tfidf__use_idf=True, total=  53.0s
[CV]  classifier__estimator__max_depth=5, tfidf__ngram_range=(1, 2), tfidf__use_idf=True, total=  54.0s
[CV]  classifier__estimator__max_depth=5, tfidf__ngram_range=(1, 2), tfidf__use_idf=True, total=  53.8s


[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:  1.1min remaining:   22.2s


[CV]  classifier__estimator__max_depth=7, tfidf__ngram_range=(1, 2), tfidf__use_idf=True, total=  53.1s
[CV]  classifier__estimator__max_depth=7, tfidf__ngram_range=(1, 2), tfidf__use_idf=True, total=  51.4s
[CV]  classifier__estimator__max_depth=7, tfidf__ngram_range=(1, 2), tfidf__use_idf=True, total=  54.4s


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  1.2min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'tfidf__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': [True], 'classifier__estimator__max_depth': [5, 7]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
y_pred = model.predict(X_test)

# Print the best parameters found by grid search
print("Best Parameters:", model.best_params_)

# Iterate through each column and print classification report
for i, column in enumerate(Y.columns):
    report = classification_report(y_test.iloc[:, i], y_pred[:, i])
    print(f"Report for {column}:")
    print(report)
    print('-' * 50)

Best Parameters: {'classifier__estimator__max_depth': 5, 'tfidf__ngram_range': (1, 2), 'tfidf__use_idf': True}
Report for offer:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244

--------------------------------------------------
Report for aid_related:
             precision    recall  f1-score   support

          0       0.60      1.00      0.75      3113
          1       0.88      0.02      0.03      2131

avg / total       0.71      0.60      0.46      5244

--------------------------------------------------
Report for medical_help:
             precision    recall  f1-score   support

          0       0.92      1.00      0.96      4822
          1       0.00      0.00      0.00       422

avg / total       0.85      0.92      0.88      5244

--------------------------------------------------
Report for medical_produ

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
# Function to extract text length from a column
def extract_text_length(column):
    return column.str.len().values.reshape(-1, 1)

# Define the pipeline components
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
classifier = MultiOutputClassifier(RandomForestClassifier())

# Create the pipeline
pipeline = Pipeline([
    ('tfidf', tfidf_vectorizer),
    ('scaler', StandardScaler(with_mean=False)),
    ('classifier', classifier)
])

# Define the parameters for grid search
parameters = {
    'classifier__estimator__max_depth': [5, 7]
}

# Perform grid search
model = GridSearchCV(pipeline, parameters, cv=3, n_jobs=-1, verbose=2)
model.fit(X_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] classifier__estimator__max_depth=5 ..............................
[CV] classifier__estimator__max_depth=5 ..............................
[CV] classifier__estimator__max_depth=5 ..............................
[CV] classifier__estimator__max_depth=7 ..............................
[CV] classifier__estimator__max_depth=7 ..............................
[CV] classifier__estimator__max_depth=7 ..............................
[CV] ............... classifier__estimator__max_depth=5, total=  33.6s
[CV] ............... classifier__estimator__max_depth=5, total=  32.9s
[CV] ............... classifier__estimator__max_depth=5, total=  36.2s


[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:   49.6s remaining:   49.6s


[CV] ............... classifier__estimator__max_depth=7, total=  37.8s
[CV] ............... classifier__estimator__max_depth=7, total=  37.7s
[CV] ............... classifier__estimator__max_depth=7, total=  38.0s


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   51.4s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
 ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'classifier__estimator__max_depth': [5, 7]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [14]:
# Predict on the testing data
y_pred = model.predict(X_test)

# Print the best parameters found by grid search
print("Best Parameters:", model.best_params_)

# Iterate through each column and print classification report
for i in range(Y.shape[1]):
    report = classification_report(y_test.iloc[:, i], y_pred[:, i])
    print(f"Report for class {i+1}:")
    print(report)
    print('-' * 50)

Best Parameters: {'classifier__estimator__max_depth': 5}
Report for class 1:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244

--------------------------------------------------
Report for class 2:
             precision    recall  f1-score   support

          0       0.60      1.00      0.75      3113
          1       0.94      0.02      0.03      2131

avg / total       0.74      0.60      0.46      5244

--------------------------------------------------
Report for class 3:
             precision    recall  f1-score   support

          0       0.92      1.00      0.96      4822
          1       0.00      0.00      0.00       422

avg / total       0.85      0.92      0.88      5244

--------------------------------------------------
Report for class 4:
             precision    recall  f1-score   support

          0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [15]:
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [21]:
import sys

def load_data(database_filepath):
    engine = create_engine('sqlite:///database.db')
    df = pd.read_sql_table('messages_classification', engine)
    df.dropna(inplace=True)
    df.dropna(inplace=True)

    X = df['message']
    Y = df.iloc[:, 4:]

    return X, Y, Y.columns


def tokenize(text):
    clean_tokens = text.apply(lambda x: x.lower()).apply(lambda x: re.sub(r'[^\w\s]', '', x))

    return clean_tokens


def build_model():
    # Function to extract text length from a column
    def extract_text_length(column):
        return column.str.len().values.reshape(-1, 1)

    # Define the pipeline components
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
    classifier = MultiOutputClassifier(RandomForestClassifier())

    # Create the pipeline
    pipeline = Pipeline([
        ('tfidf', tfidf_vectorizer),
        ('scaler', StandardScaler(with_mean=False)),
        ('classifier', classifier)
    ])

    # Define the parameters for grid search
    parameters = {
        'classifier__estimator__max_depth': [5, 7]
    }

    # Perform grid search
    model = GridSearchCV(pipeline, parameters, cv=3, n_jobs=-1, verbose=2)

    return model


def evaluate_model(model, X_test, Y_test, category_names):
    y_pred = model.predict(X_test)

    for i, category in enumerate(category_names):
        print(f"Category: {category}")
        print(classification_report(Y_test.iloc[:, i], y_pred[:, i]))


def save_model(model, model_filepath):
    with open(model_filepath, 'wb') as f:
        pickle.dump(model, f)


def main():
    if len(sys.argv) == 3:
        database_filepath, model_filepath = sys.argv[1:]
        print('Loading data...\n    DATABASE: {}'.format(database_filepath))
        X, Y, category_names = load_data(database_filepath)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

        print('Building model...')
        model = build_model()

        print('Training model...')
        model.fit(X_train, Y_train)

        print('Evaluating model...')
        evaluate_model(model, X_test, Y_test, category_names)

        print('Saving model...\n    MODEL: {}'.format(model_filepath))
        save_model(model, model_filepath)

        print('Trained model saved!')

    else:
        print('Please provide the filepath of the disaster messages database '
              'as the first argument and the filepath of the pickle file to '
              'save the model to as the second argument. \n\nExample: python '
              'train_classifier.py ../data/DisasterResponse.db classifier.pkl')


if __name__ == '__main__':
    main()

Loading data...
    DATABASE: -f
Building model...
Training model...
Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] classifier__estimator__max_depth=5 ..............................
[CV] classifier__estimator__max_depth=5 ..............................
[CV] classifier__estimator__max_depth=5 ..............................
[CV] classifier__estimator__max_depth=7 ..............................
[CV] classifier__estimator__max_depth=7 ..............................
[CV] classifier__estimator__max_depth=7 ..............................
[CV] ............... classifier__estimator__max_depth=5, total=  13.3s
[CV] ............... classifier__estimator__max_depth=5, total=  13.9s
[CV] ............... classifier__estimator__max_depth=5, total=  13.7s


[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:   19.6s remaining:   19.6s


[CV] ............... classifier__estimator__max_depth=7, total=  14.9s
[CV] ............... classifier__estimator__max_depth=7, total=  15.4s
[CV] ............... classifier__estimator__max_depth=7, total=  15.5s


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   20.5s finished


Evaluating model...
Category: related
             precision    recall  f1-score   support

          0       0.86      0.01      0.02       650
          1       0.67      1.00      0.80      1361
          2       0.00      0.00      0.00        23

avg / total       0.72      0.67      0.54      2034

Category: request
             precision    recall  f1-score   support

          0       0.65      1.00      0.79      1303
          1       0.95      0.05      0.10       731

avg / total       0.76      0.66      0.54      2034

Category: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      2032
          1       0.00      0.00      0.00         2

avg / total       1.00      1.00      1.00      2034

Category: aid_related
             precision    recall  f1-score   support

          0       0.64      1.00      0.78      1232
          1       0.95      0.12      0.22       802

avg / total       0.76      0.65      0.56     

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Trained model saved!
